<a href="https://colab.research.google.com/github/Dusein/MachineLearningTask/blob/main/12thWeekTask/CNN_For_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
from sklearn.model_selection import train_test_split

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling_type='max'):
        super(CNN, self).__init__()

        self.kernel_size = kernel_size
        self.pooling_type = pooling_type

        self.conv1 = nn.Conv2d(3, 32, kernel_size=self.kernel_size, padding=self.kernel_size // 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=self.kernel_size, padding=self.kernel_size // 2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=self.kernel_size, padding=self.kernel_size // 2)

        if self.pooling_type == 'max':
            self.pool = nn.MaxPool2d(2, 2)
        elif self.pooling_type == 'avg':
            self.pool = nn.AvgPool2d(2, 2)

        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)

        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [2]:
# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

100%|██████████| 170M/170M [00:03<00:00, 48.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Early Stopping Callback
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [7]:
# Train and evaluate function
def train_and_evaluate(kernel_size, pooling_type, optimizer_type, epochs):
    model = CNN(kernel_size=kernel_size, pooling_type=pooling_type).to(device)

    # Define optimizer
    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif optimizer_type == 'RMSProp':
        optimizer = optim.RMSprop(model.parameters(), lr=0.001)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)

    criterion = nn.CrossEntropyLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    early_stopping = EarlyStopping(patience=10)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_loss = 0.0
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = 100 * correct / total
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")

        if early_stopping(val_loss):
            print("Early stopping triggered")
            break

In [ ]:
# Hyperparameters to compare
kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
optimizers = ['SGD', 'RMSProp', 'Adam']
epochs_list = [5]
# epochs_list = [5, 50, 100, 250, 350]
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Iterate through configurations
for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer_type in optimizers:
            for epochs in epochs_list:
                print(f"\nConfiguration: Kernel={kernel_size}, Pooling={pooling_type}, Optimizer={optimizer_type}, Epochs={epochs}")
                train_and_evaluate(kernel_size, pooling_type, optimizer_type, epochs)


Configuration: Kernel=3, Pooling=max, Optimizer=SGD, Epochs=5
Epoch 1/5, Training Loss: 2.0285, Validation Loss: 1.7005, Test Accuracy: 37.43%
Epoch 2/5, Training Loss: 1.6173, Validation Loss: 1.4721, Test Accuracy: 46.32%
Epoch 3/5, Training Loss: 1.4612, Validation Loss: 1.3560, Test Accuracy: 50.28%
Epoch 4/5, Training Loss: 1.3412, Validation Loss: 1.2412, Test Accuracy: 55.06%
Epoch 5/5, Training Loss: 1.2334, Validation Loss: 1.1171, Test Accuracy: 59.58%

Configuration: Kernel=3, Pooling=max, Optimizer=RMSProp, Epochs=5
Epoch 1/5, Training Loss: 1.8109, Validation Loss: 1.5512, Test Accuracy: 43.59%
Epoch 2/5, Training Loss: 1.4608, Validation Loss: 1.2834, Test Accuracy: 53.27%
Epoch 3/5, Training Loss: 1.2888, Validation Loss: 1.1899, Test Accuracy: 57.02%
Epoch 4/5, Training Loss: 1.1629, Validation Loss: 1.1334, Test Accuracy: 59.62%
Epoch 5/5, Training Loss: 1.0759, Validation Loss: 0.9643, Test Accuracy: 65.60%

Configuration: Kernel=3, Pooling=max, Optimizer=Adam, Epoch